In [ ]:
import openpyxl
import populartimes
from googleplaces import GooglePlaces, types, lang, GooglePlacesError

YOUR_API_KEY = ''
google_places = GooglePlaces(YOUR_API_KEY)

In [ ]:
# This code opens the 'TennisCourts.xlsx' file using the openpyxl library 
# and extracts the tennis court names, longitudes, and latitudes from each row. 
# The data is then stored in separate lists for each parameter, allowing for easy access and manipulation.

xl_path_1 = "/Users/pumpedup_py/Downloads/TennisCourts.xlsx"
wb_1 = openpyxl.load_workbook(xl_path_1)
main_sheet_1 = wb_1["in"]

place_park_list = []
lat_list=[]
lng_list=[]

for i in range(2, main_sheet_1.max_row+1):
    park_name = main_sheet_1.cell(row=i, column=4).value
    lat = main_sheet_1.cell(row=i, column=5).value
    lng = main_sheet_1.cell(row=i, column=6).value
    place_park_list.append(park_name)
    lat_list.append(lat)
    lng_list.append(lng)
   

In [ ]:
# Retrieve the Google Place ID using latitude and longitude data. 
# If a Place ID cannot be found, then store the name of the location and its corresponding index
# in the list 'missed_place_index'.

place_name=[]
place_id=[]
missed_place=[]
missed_place_index=[]

for i in range(len(place_park_list)):
    query_result = google_places.nearby_search(
            lat_lng={'lat': lat_list[i], 'lng': lng_list[i]} , keyword=place_park_list[i],
            radius = 1)
    
    try:
        place_name.append(query_result.places[0].name)
        place_id.append(query_result.places[0].place_id)
    
    except:
        place_name.append(place_park_list[i])
        place_id.append("NA")
        missed_place.append(place_park_list[i])
        missed_place_index.append(i)
        pass
    

In [ ]:
# Verify whether the list contains all 870 items from the corresponding Excel file.

print(len(place_name))
print(len(place_id))
print(len(missed_place))
print(len(missed_place_index))

In [ ]:
# Store the place index that only has Place ID.

right_place_index = list(range(len(place_park_list)))
right_place_index = [i for i in right_place_index if i not in missed_place_index]

In [ ]:
# veryfy that there is 'None' value in the place name list

for i in range(len(place_park_list)):
    if place_park_list[i] is None:
        print(f"Index {i} contains None.")

In [ ]:
# This function calculates the popularity normalization factor of a place using the Populartimes API.


def get_norm(YOUR_API_KEY, place_id, i):
    # It returns a dictionary with information about the popular times of the place.
    pop=populartimes.get_id(YOUR_API_KEY, place_id[i])

    total_avg = 0
    max_norm = 0
    weeks_avg_data=[]
    
    # The popularity data is extracted from the dictionary for each day of the week.
    mon = pop['populartimes'][0]['data']
    tue = pop['populartimes'][1]['data']
    wed = pop['populartimes'][2]['data']
    thu = pop['populartimes'][3]['data']
    fri = pop['populartimes'][4]['data']
    sat = pop['populartimes'][5]['data']
    sun = pop['populartimes'][6]['data']

    # For calculating the average of popular times data.
    def list_avg(lists):
        return sum(lists)/len(lists)

    total_avg = list_avg(mon)+list_avg(tue)+list_avg(wed)+list_avg(thu)+list_avg(fri)+list_avg(sat)+list_avg(sun)

    weeks_avg_data.append(list_avg(mon))
    weeks_avg_data.append(list_avg(tue))
    weeks_avg_data.append(list_avg(wed))
    weeks_avg_data.append(list_avg(thu))
    weeks_avg_data.append(list_avg(fri))
    weeks_avg_data.append(list_avg(sat))
    weeks_avg_data.append(list_avg(sun))

    # Get the maximum popularity value of a week.
    matches_max=max(weeks_avg_data)

    # Calculate the maximum normalization factor.
    max_norm=round(matches_max/total_avg, 4)
    
    return max_norm

In [ ]:
# The first loop iterates over the indices of the places 
# for which the popularity normalization factor needs to be calculated.
# The indices are stored in the list "right_place_index".

for i in right_place_index:
    try:
        main_sheet_1.cell(row=i+2, column=8).value = place_id[i]
        main_sheet_1.cell(row=i+2, column=9).value = place_name[i]
        main_sheet_1.cell(row=i+2, column=10).value = get_norm(YOUR_API_KEY, place_id, i)
    
    # If an error occurs while attempting to get the popularity normalization factor 
    # for a particular place (e.g., due to an invalid API key), the loop moves on to the next iteration 
    # without raising an exception.
    
    except:
        pass

# The second loop iterates over the indices of the places 
# that were not found in the Populartimes API.
# The indices are stored in the list "missed_place_index".

for i in missed_place_index:
    main_sheet_1.cell(row=i+2, column=8).value = None
    main_sheet_1.cell(row=i+2, column=9).value = None
    main_sheet_1.cell(row=i+2, column=10).value = None
    
wb_1.save(xl_path_1)